In [14]:
import os
import random
import pandas as pd
import selfies as sf

from glob import glob
from tqdm.auto import tqdm
from datasets import load_dataset

In [19]:
os.makedirs('./preproc', exist_ok=True)

In [2]:
smiles = []
query = []
answer = []

smiles_val=[]
query_val=[]
answer_val=[]

In [3]:
mol_inst = load_dataset("zjunlp/Mol-Instructions", 'Molecule-oriented Instructions')

In [4]:
mol_inst.keys()

dict_keys(['description_guided_molecule_design', 'forward_reaction_prediction', 'molecular_description_generation', 'property_prediction', 'reagent_prediction', 'retrosynthesis'])

In [5]:
for data in tqdm(mol_inst['molecular_description_generation']):
    smiles.append(sf.decoder(data['input']))
    query.append(data['instruction'])
    answer.append(data['output'])

  0%|          | 0/331261 [00:00<?, ?it/s]

In [6]:
for data in tqdm(mol_inst['property_prediction']):
    smiles.append(sf.decoder(data['input']))
    query.append(data['instruction'])
    answer.append(data['output'])

  0%|          | 0/401229 [00:00<?, ?it/s]

In [7]:
for data in tqdm(mol_inst['retrosynthesis']):
    smiles.append(sf.decoder(data['input']))
    query.append(data['instruction'])
    output_sfs = data['output'].split('.')
    answer.append('.'.join(list(map(sf.decoder, output_sfs))))

  0%|          | 0/143535 [00:00<?, ?it/s]

In [8]:
# from huggingface_hub import snapshot_download
# snapshot_download(repo_id="haitengzhao/molecule_property_instruction", repo_type="dataset")

In [9]:
bace_parquet = glob('C:\\Users\\dust\\.cache\\huggingface\\hub\\datasets--haitengzhao--molecule_property_instruction\\snapshots\\aad5c7578e811e1614be9430095de0c431485cf3\\data\\bace*')[0]

In [10]:
bace_df = pd.read_parquet(bace_parquet)

In [11]:
for idx, row in tqdm(bace_df.iterrows(), total=len(bace_df)):
    if row['split'] == 'train':
        for q in row['text']:
            smiles.append(row['graph'])
            query.append(q)
            answer.append(row['label'])
    elif row['split'] != 'train':
        for q in row['text']:
            smiles_val.append(row['graph'])
            query_val.append(q)
            answer_val.append(row['label'])

  0%|          | 0/1513 [00:00<?, ?it/s]

In [12]:
bbbp_parquet = glob('C:\\Users\\dust\\.cache\\huggingface\\hub\\datasets--haitengzhao--molecule_property_instruction\\snapshots\\aad5c7578e811e1614be9430095de0c431485cf3\\data\\bbbp*')[0]

In [13]:
bbbp_df = pd.read_parquet(bbbp_parquet)

In [14]:
for idx, row in tqdm(bbbp_df.iterrows(), total=len(bbbp_df)):
    if row['split'] == 'train':
        for q in row['text']:
            smiles.append(row['graph'])
            query.append(q)
            answer.append(row['label'])
    elif row['split'] != 'train':
        for q in row['text']:
            smiles_val.append(row['graph'])
            query_val.append(q)
            answer_val.append(row['label'])

  0%|          | 0/2039 [00:00<?, ?it/s]

In [15]:
chembl_parquets = glob('C:\\Users\\dust\\.cache\\huggingface\\hub\\datasets--haitengzhao--molecule_property_instruction\\snapshots\\aad5c7578e811e1614be9430095de0c431485cf3\\data\\chembl*')
for chembl_parquet in tqdm(chembl_parquets, total=len(chembl_parquets)):
    chembl_df = pd.read_parquet(chembl_parquet)
    for idx, row in hembl_df.iterrows():
        if row['split'] == 'train':
            for q in row['text']:
                smiles.append(row['graph'])
                query.append(q)
                answer.append(row['label'])
        elif row['split'] != 'train':
            for q in row['text']:
                smiles_val.append(row['graph'])
                query_val.append(q)
                answer_val.append(row['label'])

  0%|          | 0/954974 [00:00<?, ?it/s]

  0%|          | 0/954974 [00:00<?, ?it/s]

  0%|          | 0/954974 [00:00<?, ?it/s]

  0%|          | 0/954974 [00:00<?, ?it/s]

  0%|          | 0/954974 [00:00<?, ?it/s]

...

  0%|          | 0/954973 [00:00<?, ?it/s]

  0%|          | 0/954973 [00:00<?, ?it/s]

  0%|          | 0/954973 [00:00<?, ?it/s]

  0%|          | 0/954973 [00:00<?, ?it/s]

  0%|          | 0/180229 [00:00<?, ?it/s]

In [16]:
esol_parquet = glob('C:\\Users\\dust\\.cache\\huggingface\\hub\\datasets--haitengzhao--molecule_property_instruction\\snapshots\\aad5c7578e811e1614be9430095de0c431485cf3\\data\\esol*')[0]

In [17]:
esol_df = pd.read_parquet(esol_parquet)

In [18]:
for idx, row in tqdm(esol_df.iterrows(), total=len(esol_df)):
    if row['split'] == 'train':
        for q in row['text']:
            smiles.append(row['graph'])
            query.append(q)
            answer.append(row['label'])
    elif row['split'] != 'train':
        for q in row['text']:
            smiles_val.append(row['graph'])
            query_val.append(q)
            answer_val.append(row['label'])

  0%|          | 0/1128 [00:00<?, ?it/s]

In [19]:
lipo_parquet = glob('C:\\Users\\dust\\.cache\\huggingface\\hub\\datasets--haitengzhao--molecule_property_instruction\\snapshots\\aad5c7578e811e1614be9430095de0c431485cf3\\data\\lipo*')[0]
lipo_df = pd.read_parquet(lipo_parquet)

In [20]:
for idx, row in tqdm(lipo_df.iterrows(), total=len(lipo_df)):
    if row['split'] == 'train':
        for q in row['text']:
            smiles.append(row['graph'])
            query.append(q)
            answer.append(row['label'])
    elif row['split'] != 'train':
        for q in row['text']:
            smiles_val.append(row['graph'])
            query_val.append(q)
            answer_val.append(row['label'])

  0%|          | 0/4200 [00:00<?, ?it/s]

In [21]:
freesolv_parquet = glob('C:\\Users\\dust\\.cache\\huggingface\\hub\\datasets--haitengzhao--molecule_property_instruction\\snapshots\\aad5c7578e811e1614be9430095de0c431485cf3\\data\\freesolv*')[0]
freesolv_df = pd.read_parquet(freesolv_parquet)

In [22]:
for idx, row in tqdm(freesolv_df.iterrows(), total=len(freesolv_df)):
    if row['split'] == 'train':
        for q in row['text']:
            smiles.append(row['graph'])
            query.append(q)
            answer.append(row['label'])
    elif row['split'] != 'train':
        for q in row['text']:
            smiles_val.append(row['graph'])
            query_val.append(q)
            answer_val.append(row['label'])

  0%|          | 0/642 [00:00<?, ?it/s]

In [23]:
hiv_parquet = glob('C:\\Users\\dust\\.cache\\huggingface\\hub\\datasets--haitengzhao--molecule_property_instruction\\snapshots\\aad5c7578e811e1614be9430095de0c431485cf3\\data\\hiv*')[0]
hiv_df = pd.read_parquet(hiv_parquet)

In [24]:
for idx, row in tqdm(hiv_df.iterrows(), total=len(hiv_df)):
    if row['split'] == 'train':
        for q in row['text']:
            smiles.append(row['graph'])
            query.append(q)
            answer.append(row['label'])
    elif row['split'] != 'train':
        for q in row['text']:
            smiles_val.append(row['graph'])
            query_val.append(q)
            answer_val.append(row['label'])

  0%|          | 0/41127 [00:00<?, ?it/s]

In [25]:
muv_parquets = glob('C:\\Users\\dust\\.cache\\huggingface\\hub\\datasets--haitengzhao--molecule_property_instruction\\snapshots\\aad5c7578e811e1614be9430095de0c431485cf3\\data\\muv*')

In [26]:
for muv_parquet in muv_parquets:
    muv_df = pd.read_parquet(muv_parquet)
    for idx, row in tqdm(muv_df.iterrows(), total=len(muv_df)):
        if row['split'] == 'train':
            for q in row['text']:
                smiles.append(row['graph'])
                query.append(q)
                answer.append(row['label'])
        elif row['split'] != 'train':
            for q in row['text']:
                smiles_val.append(row['graph'])
                query_val.append(q)
                answer_val.append(row['label'])

  0%|          | 0/124943 [00:00<?, ?it/s]

  0%|          | 0/124943 [00:00<?, ?it/s]

In [27]:
tox21_parquet = glob('C:\\Users\\dust\\.cache\\huggingface\\hub\\datasets--haitengzhao--molecule_property_instruction\\snapshots\\aad5c7578e811e1614be9430095de0c431485cf3\\data\\tox21*')[0]
tox21_df = pd.read_parquet(tox21_parquet)

In [28]:
for idx, row in tqdm(tox21_df.iterrows(), total=len(tox21_df)):
    if row['split'] == 'train':
        for q in row['text']:
            smiles.append(row['graph'])
            query.append(q)
            answer.append(row['label'])
    elif row['split'] != 'train':
        for q in row['text']:
            smiles_val.append(row['graph'])
            query_val.append(q)
            answer_val.append(row['label'])

  0%|          | 0/77946 [00:00<?, ?it/s]

In [29]:
toxcast_parquets = glob('C:\\Users\\dust\\.cache\\huggingface\\hub\\datasets--haitengzhao--molecule_property_instruction\\snapshots\\aad5c7578e811e1614be9430095de0c431485cf3\\data\\toxcast*')

In [30]:
for toxcast_parquet in toxcast_parquets:
    toxcast_df = pd.read_parquet(toxcast_parquet)
    for idx, row in tqdm(toxcast_df.iterrows(), total=len(toxcast_df)):
        if row['split'] == 'train':
            for q in row['text']:
                smiles.append(row['graph'])
                query.append(q)
                answer.append(row['label'])
        elif row['split'] != 'train':
            for q in row['text']:
                smiles_val.append(row['graph'])
                query_val.append(q)
                answer_val.append(row['label'])

  0%|          | 0/372603 [00:00<?, ?it/s]

  0%|          | 0/372603 [00:00<?, ?it/s]

  0%|          | 0/372603 [00:00<?, ?it/s]

  0%|          | 0/372603 [00:00<?, ?it/s]

In [31]:
cyp450_parquet = glob('C:\\Users\\dust\\.cache\\huggingface\\hub\\datasets--haitengzhao--molecule_property_instruction\\snapshots\\aad5c7578e811e1614be9430095de0c431485cf3\\data\\cyp450*')[0]
cyp450_df = pd.read_parquet(cyp450_parquet)

In [32]:
for idx, row in tqdm(cyp450_df.iterrows(), total=len(cyp450_df)):
    if row['split'] == 'train':
        for q in row['text']:
            smiles.append(row['graph'])
            query.append(q)
            answer.append(row['label'])
    elif row['split'] != 'train':
        for q in row['text']:
            smiles_val.append(row['graph'])
            query_val.append(q)
            answer_val.append(row['label'])

  0%|          | 0/53178 [00:00<?, ?it/s]

In [33]:
pcba_parquets = glob('C:\\Users\\dust\\.cache\\huggingface\\hub\\datasets--haitengzhao--molecule_property_instruction\\snapshots\\aad5c7578e811e1614be9430095de0c431485cf3\\data\\pcba*')

In [34]:
for pcba_parquet in tqdm(pcba_parquets, total=len(pcba_parquets)):
    pcba_df = pd.read_parquet(pcba_parquet)
    for idx, row in pcba_df.iterrows():
        if row['split'] == 'train':
            for q in row['text']:
                smiles.append(row['graph'])
                query.append(q)
                answer.append(row['label'])
        elif row['split'] != 'train':
            for q in row['text']:
                smiles_val.append(row['graph'])
                query_val.append(q)
                answer_val.append(row['label'])

  0%|          | 0/773118 [00:00<?, ?it/s]

  0%|          | 0/773118 [00:00<?, ?it/s]

  0%|          | 0/773118 [00:00<?, ?it/s]

  0%|          | 0/773118 [00:00<?, ?it/s]

  0%|          | 0/773118 [00:00<?, ?it/s]

...

  0%|          | 0/773117 [00:00<?, ?it/s]

  0%|          | 0/773117 [00:00<?, ?it/s]

  0%|          | 0/773117 [00:00<?, ?it/s]

  0%|          | 0/773117 [00:00<?, ?it/s]

  0%|          | 0/773117 [00:00<?, ?it/s]

In [35]:
chembl_drug_inst = load_dataset("alxfgh/ChEMBL_Drug_Instruction_Tuning")

In [36]:
for data in tqdm(chembl_drug_inst['train']):
    smiles.append(data['SMILES'])
    query.append(data['Question'])
    answer.append(data['Answer'])

  0%|          | 0/129699 [00:00<?, ?it/s]

In [37]:
pubchem_drug_inst = load_dataset("alxfgh/PubChem_Desc_DrugChat")

In [38]:
for data in tqdm(pubchem_drug_inst['train']):
    smiles.append(data['SMILES'])
    query.append(random.choice(['Describe the molecule.', 'Give me a description of the molecule.', 'What is the description of the molecule?', 'Explain the molecule', 'Information about the molecule']))
    answer.append(data['Descriptor'])

  0%|          | 0/13818 [00:00<?, ?it/s]

In [39]:
instruction_df = pd.DataFrame(columns=['smiles', 'query', 'answer'])
instruction_df['smiles'] = smiles
instruction_df['query'] = query
instruction_df['answer'] = answer

instruction_df.to_csv('./preproc/instruction.csv', index=False)

In [40]:
instruction_val_df = pd.DataFrame(columns=['smiles', 'query', 'answer'])
instruction_val_df['smiles'] = smiles_val
instruction_val_df['query'] = query_val
instruction_val_df['answer'] = answer_val

instruction_val_df.to_csv('./preproc/instruction_val.csv', index=False)

In [41]:
smiles=[]
query=[]
answer=[]

smiles_val=[]
query_val=[]
answer_val=[]

In [42]:
zinc_cannon = load_dataset("sagawa/ZINC-canonicalized")

In [43]:
for data in tqdm(zinc_cannon['train']):
    smiles.append(data['smiles'])
    query.append(random.choice(['What is the SMILES of this molecule?', 'What is the SMILES string of this molecule?', 'What is the SMILES string of this compound?', 'What is the SMILES of this compound?', 'Give me the SMILES of this molecule.', 'Give me the SMILES of this compound.', 'Give me the SMILES string of this molecule.', 'Give me the SMILES string of this compound.']))
    answer.append(data['smiles'])
for data in tqdm(zinc_cannon['validation']):
    smiles_val.append(data['smiles'])
    query_val.append(random.choice(['What is the SMILES of this molecule?', 'What is the SMILES string of this molecule?', 'What is the SMILES string of this compound?', 'What is the SMILES of this compound?', 'Give me the SMILES of this molecule.', 'Give me the SMILES of this compound.', 'Give me the SMILES string of this molecule.', 'Give me the SMILES string of this compound.']))
    answer_val.append(data['smiles'])

  0%|          | 0/20693269 [00:00<?, ?it/s]

  0%|          | 0/2299253 [00:00<?, ?it/s]

In [ ]:
pretrain_df = pd.DataFrame(columns=['smiles', 'query', 'answer'])
pretrain_df['smiles'] = smiles
pretrain_df['query'] = query
pretrain_df['answer'] = answer

pretrain_df.sample(100).to_csv('./preproc/pretrain_sample.csv', index=False)
pretrain_df.to_csv('./preproc/pretrain.csv', index=False)

In [ ]:
pretrain_val_df = pd.DataFrame(columns=['smiles', 'query', 'answer'])
pretrain_val_df['smiles'] = smiles_val
pretrain_val_df['query'] = query_val
pretrain_val_df['answer'] = answer_val

pretrain_val_df.sample(100).to_csv('./preproc/pretrain_val_sample.csv', index=False)
pretrain_val_df.to_csv('./preproc/pretrain_val.csv', index=False)